In [ ]:
pwd

In [ ]:
cd ..


In [ ]:
import gargaml
from gargaml import *
from gargaml import Data

In [ ]:
X, y  = Data().Load.titanic(True, precleaning=True)
X

In [ ]:
sns.pairplot(df, hue="survived", corner=True)

In [ ]:
px.box(df, x="survived", y="fare")

In [ ]:
px.box(df, x="survived", y="pclass")

In [ ]:
px.box(df, x="survived", y="age", color="sex")

In [ ]:
X.drop(columns="passengerid", inplace=True)
X.sample(10)

In [ ]:
y

In [ ]:
y.value_counts(normalize=True).round(2)

In [ ]:
X.info()

In [ ]:
msno.matrix(X)

In [ ]:
# # viert pret à 0



# # virer total amont/anuity  / target => NAN
# # virer total amont/anuity  => 0 




# # feat eng.
# df_amout_rate = df.toal_amount / df.annuity
# df_salary_rate = df.toal_amount / df.salary 
# df_annuity_rate =  df.annuity / df.Salary 

In [ ]:
X["fare_by_age"] = X.fare/ X.age
X["fare_by_pclass"] = X.fare/X.pclass 
X["fare_null"] = (X.fare == 0).astype(int)
X["age_is_nan"] = X.age.isna().astype(int)
X.describe()

In [ ]:
X

In [ ]:
detector = EllipticEnvelope()
tmp = X.select_dtypes(np.number)
tmp =KNNImputer().fit_transform(tmp)
ee = detector.fit_predict(tmp)
ee = pd.Series(ee, name="is_outlier")

In [ ]:
ee.value_counts(normalize=True).round(2
                                      )

In [ ]:
ee = ee.apply(lambda i : False if i==1 else True)
ee

In [ ]:
ee.value_counts()

In [ ]:
X["algorithmic_outlier"] = ee.values.astype(int)
X

In [ ]:
X_1 = X.copy()
X_1.describe().astype(int)

In [ ]:
X_2 = X.loc[~ee.values]
X_2

In [ ]:
X_2.describe().astype(int)

In [ ]:
del X_1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
X_train.isna().mean().round(4)

In [ ]:
y_train

In [ ]:
imputer = KNNImputer()

cols = X_train.select_dtypes(include=np.number).columns
display(cols)

imputer.fit(X_train.select_dtypes(include=np.number))
X_train = imputer.transform(X_train.select_dtypes(include=np.number))

# X_train = imputer.fit_transform(X_train.select_dtypes(include=np.number))
X_test = imputer.transform(X_test.select_dtypes(include=np.number))
X_train





In [ ]:
X_train.shape

In [ ]:
X_train = pd.DataFrame(X_train, columns=cols)
X_test = pd.DataFrame(X_test, columns=cols)
X_train

In [ ]:
X_train.isna().sum()

In [ ]:
y_train.value_counts(normalize=True).round(2)

In [ ]:
y_test.value_counts(normalize=True).round(2)

In [ ]:
estimator = RandomForestClassifier()
# estimator = LogisticRegression()
estimator.fit(X_train, y_train)

In [ ]:
y_pred_train = estimator.predict(X_train)
y_pred_train

In [ ]:
y_train

In [ ]:
estimator.score(X_train, y_train).round(2)

In [ ]:
estimator.score(X_test, y_test).round(2)


In [ ]:
X_train

In [ ]:
estimator.predict(X_train)

In [ ]:
X_train.iloc[0]
p0 = X_train.iloc[0]
p0

In [ ]:
estimator.predict(p0.reshape(-1, 1))

In [ ]:
p0.values

In [ ]:
p0.values.reshape(-1, 1)

In [ ]:
estimator.predict(p0.values.reshape(-1, 1))

In [ ]:
X_train.iloc[0]

In [ ]:
X_train.iloc[:1]

In [183]:
p0 = X_train.iloc[:1]
p0

,passengerid,pclass,age,sibsp,parch,fare,fare_by_age,fare_by_pclass,fare_null,age_is_nan,algorithmic_outlier
0,271.0,1.0,20.4,0.0,0.0,31.0,3.635048,31.0,0.0,1.0,0.0


In [184]:
pred_p0 = estimator.predict_proba(p0)
pred_p0[0]

array([0.86, 0.14])

In [185]:
feat_importance = pd.Series(estimator.coef_[0], index=X_train.columns)
feat_importance.round(2).sort_values(ascending=False)

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

In [186]:
estimator.intercept_[0]

AttributeError: 'RandomForestClassifier' object has no attribute 'intercept_'

In [187]:
feat_importance = pd.Series(estimator.feature_importances_, index=X_train.columns)
feat_importance = feat_importance.round(2).sort_values(ascending=False)
feat_importance

passengerid            0.20
fare_by_age            0.19
age                    0.16
fare_by_pclass         0.16
fare                   0.14
pclass                 0.06
sibsp                  0.04
parch                  0.03
age_is_nan             0.02
algorithmic_outlier    0.01
fare_null              0.00
dtype: float64

In [188]:
feat_importance.to_dict()

{'passengerid': 0.2,
 'fare_by_age': 0.19,
 'age': 0.16,
 'fare_by_pclass': 0.16,
 'fare': 0.14,
 'pclass': 0.06,
 'sibsp': 0.04,
 'parch': 0.03,
 'age_is_nan': 0.02,
 'algorithmic_outlier': 0.01,
 'fare_null': 0.0}

In [190]:
import shap

In [196]:
explainer = shap.Explainer(estimator)


In [197]:
X_train.iloc[:1]

,passengerid,pclass,age,sibsp,parch,fare,fare_by_age,fare_by_pclass,fare_null,age_is_nan,algorithmic_outlier
0,271.0,1.0,20.4,0.0,0.0,31.0,3.635048,31.0,0.0,1.0,0.0


In [199]:
shap_values = explainer.shap_values(X_train.iloc[:1])
# shap_values ==> comprenne la structure de cette objet
# [0 {} , 1 {}] ???

AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:

Chaque point => ROUTE

1 / regle le pb de predict / predict proba 
2 / Comprenne objet de shap values ==> fabriquer un dictionnaire => envoyer depuis api (json)
3 / Route descibre => X_train.mean().to_dict() => route

==> derniere partie de API ==> METTRE API "en ligne" ==> heroku ==> git / git hub (logiciel qui héberge le code)

NE PRENDS QUE 10 % de daats => SI API avec 100 lignes (cest ok)

In [ ]:
scaler = StandardScaler()  # QuantileTransformer(n_quantiles=100)
# scaler = MinMaxScaler()  # QuantileTransformer(n_quantiles=100)

# scaler.fit(X_train)
# X_train_sca = scaler.transform(X_train)
X_train_sca = scaler.fit_transform(X_train)
X_test_sca = scaler.transform(X_test)

In [ ]:
X_test_sca

In [ ]:
X_train_sca = pd.DataFrame(X_train_sca, columns=cols)
X_test_sca = pd.DataFrame(X_test_sca, columns=cols)
X_train_sca

In [ ]:
X_train.describe().round(1)

In [ ]:
X_train_sca.describe().round(1)

In [ ]:
from IPython.display import display

estimator = LinearRegression()
estimator.fit(X_train_sca, y_train)

display(estimator.score(X_train_sca, y_train).round(2))
display(estimator.score(X_test_sca, y_test).round(2))

In [ ]:
# estimator = LogisticRegression()
# estimator.fit(X_train_sca, y_train)

# display(estimator.score(X_train_sca, np.log1p(y_train).astype(int)))
# display(estimator.score(X_test_sca, np.log1p(y_test).astype(int)))

In [ ]:
y_train

In [ ]:
# np.log1p(y_train).astype(int).value_counts()

In [ ]:
grid = GridSearchCV(
    LogisticRegression(),
    {},
    cv=10, # StratifiedShuffleSplit(n_splits=10, test_size=0.3)
    n_jobs=4,
    verbose=2,
    return_train_score=True,
)
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
display(grid.score(X_train, y_train))
display(grid.score(X_test, y_test))

In [ ]:
display(grid.best_estimator_.score(X_train, y_train))
display(grid.best_estimator_.score(X_test, y_test))

In [ ]:
grid.cv_results_

In [ ]:
res = pd.DataFrame(grid.cv_results_).round(2)
res

In [ ]:
cols = [i for i in res.columns if "split" not in i]
res = res.loc[:, cols]
res

In [ ]:
def resultize(grid):
    """from a fited grid search build / return a fancy dataframe of results"""

    res = pd.DataFrame(grid.cv_results_).round(2)
    cols = [i for i in res.columns if "split" not in i]
    res = res.loc[:, cols]

    res = res.sort_values("mean_test_score", ascending=False)

    return res

In [ ]:
blabla = resultize(grid)
blabla

In [ ]:
pipe = Pipeline(
    [
        # ("sampler", RandomUnderSampler()),
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("estimator", LogisticRegression()),
    ]
)

pipe

In [ ]:
grid = GridSearchCV(pipe, {}, cv=10, n_jobs=-1, verbose=2, return_train_score=True)
grid.fit(X.select_dtypes(include=np.number), y)

In [ ]:
resultize(grid)

In [ ]:
li = [
    # ("sampler", RandomUnderSampler()),
    ("imputer", KNNImputer()),
    ("scaler", StandardScaler()),
    ("estimator", LogisticRegression()),
]

pipe = Pipeline(
    li
)
pipe

In [ ]:
param_grid = {


    # "estimator": [LogisticRegression()],
    


    # "sampler": ["passthrough", RandomUnderSampler()],
    
    "imputer": [
        KNNImputer(),
        SimpleImputer(strategy="mean"),
        SimpleImputer(strategy="median"),
    ],
}
param_grid

In [ ]:
# def cv():
#     return StratifiedShuffleSplit(n_splits=10, test_size=0.3)

In [ ]:
N_JOBS = -1
VERBOSE = 1
CV = 10

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)
grid.fit(X.select_dtypes(include=np.number), y)
resultize(grid)

In [ ]:
param_grid = {
    "estimator": [LogisticRegression(), ], # RandomForestClassifier()
    #

    # "sampler": ["passthrough", RandomUnderSampler()],
    
    "imputer": [KNNImputer(), SimpleImputer()],
    
    "scaler": [
        StandardScaler(),
        Normalizer(),
        QuantileTransformer(n_quantiles=100),
        MinMaxScaler(),
        "passthrough",
    ],
}
param_grid

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(X.select_dtypes(include=np.number), y)
resultize(grid)

In [ ]:
param_grid = {
    # "imputer"  : [KNNImputer(), SimpleImputer()],
    # "scaler" : [StandardScaler(), Normalizer(), QuantileTransformer(n_quantiles=100), "passthrough"],  # MinMaxScaler()
    "estimator": [
        DummyClassifier(),
        LogisticRegression(),
        KNeighborsClassifier(),
        RandomForestClassifier(),
    ]
}
param_grid

In [ ]:
grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=CV, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(X.select_dtypes(include=np.number), y)
resultize(grid)

In [ ]:
param_grid = {
    
    # "sampler": ["passthrough", RandomUnderSampler()],

    "imputer": [KNNImputer(), SimpleImputer()],

    "scaler": [
        StandardScaler(),
        Normalizer(),
        QuantileTransformer(n_quantiles=100),
        "passthrough",
    ],  # MinMaxScaler()
    
    "estimator": [
        LogisticRegression(),
        KNeighborsClassifier(),
        RandomForestClassifier(),
    ],  # DummyClassifier()
}
param_grid

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)
grid.fit(X.select_dtypes(include=np.number), y)
resultize(grid).sort_values("mean_test_score", ascending=False).head()

In [ ]:
resultize(grid).sort_values("mean_test_score", ascending=False)

In [ ]:
pd.get_dummies(X)

In [ ]:
pipe

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)
grid.fit(pd.get_dummies(X), y)
resultize(grid).head()

In [ ]:
param_grid = {
    "sampler": [
        "passthrough",
    ],  # RandomUnderSampler()
    "imputer": [
        KNNImputer(),
    ],
    "scaler": [
        StandardScaler(),
        Normalizer(),
        QuantileTransformer(n_quantiles=100),
        "passthrough",
    ],
    "estimator": [
        KNeighborsClassifier(),
        RandomForestClassifier(),
        XGBClassifier(),
    ],  # LogisticRegression(),
}

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv(),
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)
grid.fit(pd.get_dummies(X), y)
resultize(grid).head()

In [ ]:
param_grid = {
    # "sampler": [
    #     "passthrough",
    # ],  # RandomUnderSampler()
    "imputer": [
        KNNImputer(),
    ],
    "scaler": [
        StandardScaler(),
        Normalizer(),
        QuantileTransformer(n_quantiles=100),
        "passthrough",
    ],
    "estimator": [
        KNeighborsClassifier(),
    ],  # LogisticRegression(),

    "estimator__n_neighbors" : [3, 5, 7, 10, 15, 20],
    "imputer__n_neighbors" :  [3, 5, 7, 10, 15, 20],
}
param_grid

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)
grid.fit(pd.get_dummies(X), y)
resultize(grid).head()

In [ ]:
sns.displot(X.age)

In [ ]:
X.age.skew().round(1)

In [ ]:
X.fare.skew()

In [ ]:
sns.displot(X.sibsp)

In [ ]:
for col in X.select_dtypes(np.number).columns : 
    
    txt = f"{col} ==> {X[col].skew():.2f}"
    print(txt)

In [ ]:
DISPLAY = False 

if DISPLAY : 
    sns.pairplot(X, corner=True)

In [ ]:
sns.displot(X.fare)

In [ ]:
sns.displot(np.log1p(X.fare))

In [ ]:
X["equipage"] = (X.fare ==0).astype(int)
X

In [ ]:
X.loc[X.equipage == 1]

In [ ]:
X_transf = X.copy()
X_transf["fare"] = np.log1p(X_transf["fare"])
X_transf

In [ ]:
X_transf["sex"] = X_transf["sex"].apply(lambda i : 1 if i =="male" else 0)
X_transf

In [ ]:
param_grid = {
    "sampler": [
        "passthrough",
    ],  # RandomUnderSampler()
    "imputer": [
        KNNImputer(),
    ],
    "scaler": [
        StandardScaler(),
        Normalizer(),
        QuantileTransformer(n_quantiles=100),
        "passthrough",
    ],
    "estimator": [
        XGBClassifier(), 
        RandomForestClassifier(), 
        KNeighborsClassifier(),
    ],  # LogisticRegression(),

    # "estimator__n_neighbors" : [3, 5, 7, 10, 15, 20],
    # "imputer__n_neighbors" :  [3, 5, 7, 10, 15, 20],
}
param_grid

In [ ]:
pipe

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv(),
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(pd.get_dummies(X), y)
resultize(grid).head()

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv(),
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(pd.get_dummies(X_transf), y)
resultize(grid).head()

In [ ]:
def resultize(grid, log_target=0, transf=0, features=""):
    """ """

    res = pd.DataFrame(grid.cv_results_).round(2)
    cols = [i for i in res.columns if "split" not in i]
    res = res.loc[:, cols]

    ######################
    ######################

    res["log_target"] = log_target
    res["transf"] = transf
    res["features"] = features

    ######################
    ######################

    return res.sort_values("mean_test_score", ascending=False)

In [ ]:
RES = pd.DataFrame()

In [ ]:
param_grid

In [ ]:
pipe

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=CV,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(pd.get_dummies(X), y)
res = resultize(grid, log_target=0, transf=0, features="dummies")


RES = pd.concat([RES, res], axis=0, ignore_index=True)
res.head()

In [ ]:
RES

In [ ]:
RES.iloc[:, -3:].nunique()

In [ ]:
grid = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=cv(),
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(pd.get_dummies(X_transf), y)
res = resultize(grid, log_target=0, transf=0, features="dummies")


RES = pd.concat([RES, res], axis=0, ignore_index=True)
res.head()

In [ ]:
grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(X.select_dtypes(include=np.number), y)

res = resultize(grid, log_target=0, transf_fare=0, features="only_num")


RES = pd.concat([RES, res], axis=0, ignore_index=True)
res.head()

In [ ]:
# grid = GridSearchCV(
#     pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
# )
# _y = np.log1p(y).astype(int)
# grid.fit(X.select_dtypes(include=np.number), _y)

# res = resultize(grid, log_target=1, transf_fare=0, features="only_num")


# RES = pd.concat([RES, res], axis=0, ignore_index=True)
# res.head()

In [ ]:
RES.iloc[:, -3:].nunique()

In [ ]:
RES.transf.value_counts()

In [ ]:
# RandomForestClassifier()	KNNImputer()	"passthrough"  "0"	"1"	"dummies"

In [ ]:
pca_pipe = Pipeline(
    [    
        # ("sampler", RandomUnderSampler()),
        # ("selector", ColumnSelector(threshold=0.33)),
        # ("preprocessor", preprocessor),
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("reductor", PCA()), 
        ("estimator", LogisticRegression()),
    ]
)


In [ ]:
param_grid = {
    # "sampler": [
    #     "passthrough",
    # ],  # RandomUnderSampler()

    "imputer": [
        KNNImputer()
    ],

    "reductor": [PCA(svd_solver = 'full')],

    "scaler": [
        StandardScaler(),
        QuantileTransformer(n_quantiles=100),
        "passthrough",
    ],         # Normalizer(),

    "estimator": [
        # XGBClassifier(), 
        # RandomForestClassifier(), 
        KNeighborsClassifier(),
    ],  # LogisticRegression(),

    "reductor__n_components" :    [0.85, 0.9, 0.95, 0.99], # 0.5, 0.7, 0.8

    "estimator__n_neighbors" : [3, 5, 7, 10, 15, 20],
    # "imputer__n_neighbors" :  [3, 5, 7, 10, 15, 20],
}
param_grid

In [ ]:
grid = GridSearchCV(
    pca_pipe,
    param_grid=param_grid,
    cv=10,
    n_jobs=N_JOBS,
    verbose=VERBOSE,
    return_train_score=True,
)

grid.fit(pd.get_dummies(X), y)
res = resultize(grid, log_target=0, transf=0, features="dummies")
RES = pd.concat([RES, res], axis=0, ignore_index=True)
display(res.head())

grid.fit(pd.get_dummies(X_transf), y)
res = resultize(grid, log_target=0, transf=1, features="dummies")
RES = pd.concat([RES, res], axis=0, ignore_index=True)

display(res.head())

In [ ]:
# param_grid = {
#     "estimator__n_estimators": [100, 150, 200, 300, 400, 500],
#     "imputer": [
#         KNNImputer(),
#     ],
#     "scaler": [
#         "passthrough",
#     ],
#     "estimator": [
#         RandomForestClassifier(),
#     ],
# }
# param_grid

In [ ]:
grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(X.select_dtypes(include=np.number), y)
res = resultize(grid, log_target=0, transf=0, features="only_num")
RES = pd.concat([RES, res], axis=0, ignore_index=True)
display(res.head())

grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(pd.get_dummies(X), y)
res = resultize(grid, log_target=0, transf=0, features="dummies")
RES = pd.concat([RES, res], axis=0, ignore_index=True)
display(res.head())


grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(X_transf.select_dtypes(include=np.number), y)
res = resultize(grid, log_target=0, transf=1, features="only_num")
RES = pd.concat([RES, res], axis=0, ignore_index=True)
display(res.head())

grid = GridSearchCV(
    pipe, param_grid=param_grid, cv=10, n_jobs=-1, verbose=1, return_train_score=True
)
grid.fit(pd.get_dummies(X_transf), y)
res = resultize(grid, log_target=0, transf=1, features="dummies")
RES = pd.concat([RES, res], axis=0, ignore_index=True)
display(res.head())

In [ ]:
RES = RES.loc[RES.log_target == 0]
RES.sort_values("mean_test_score", ascending=False)

In [ ]:
df = Data.Load.food(False)
DF = df.copy()
df

In [ ]:
df = DF.copy()

In [ ]:
df.product_name = df.product_name.fillna("unknown")


In [ ]:

df.loc[df.product_name.str.lower().str.contains("nutell")]

In [ ]:
cols = ['energy_100g', 'fat_100g',
       'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g', ]
df = df.dropna(axis=0, how="all", subset=cols)


cols = ["product_name", 'additives_n', 
'ingredients_from_palm_oil_n', 'nutrition_grade_fr', 'pnns_groups_1',
'energy_100g', 'fat_100g',
'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g', 'fiber_100g',
'proteins_100g', 'salt_100g', 'nutrition-score-fr_100g', ]




df.product_name = df.product_name.fillna("unknown")
df = df.loc[:, cols]
df

In [ ]:
df.loc[df.product_name.str.lower().str.contains("nutell")]

In [ ]:
x= df.iloc[:, -9:-1]
x

In [ ]:
x = SimpleImputer(strategy="median").fit_transform(x)
ee = EllipticEnvelope().fit_predict(x)
# pd.Series(ee).value_counts()


ee= [True if i == 1 else False for i in ee ]
pd.Series(ee).value_counts().round(2)

In [ ]:
len(ee)

In [ ]:
len(df)

In [ ]:
df = df.loc[ ee , :  ]
df

In [ ]:
norm = df.loc[df.nutrition_grade_fr.notna()]
norm.describe().round(1)

In [ ]:
df.nutrition_grade_fr  = df.nutrition_grade_fr.fillna('zz')

In [ ]:
_df = df.copy()
_df = _df.sort_values("nutrition_grade_fr")
_df["is_zz"]= _df.nutrition_grade_fr=="zz"

px.box(_df, x="pnns_groups_1", y ="energy_100g")

In [ ]:

px.box(_df, x="is_zz", y ="energy_100g")

In [ ]:
px.box(_df, x="is_zz", y ="sugars_100g")

In [ ]:
px.box(_df, x="is_zz", y ="fat_100g")

In [ ]:
px.box(_df, color="is_zz", y ="energy_100g" ,x="pnns_groups_1")

In [ ]:
px.box(_df, color="is_zz", y ="sugars_100g" ,x="pnns_groups_1")

In [ ]:
px.box(_df, color="is_zz", y ="fat_100g" ,x="pnns_groups_1")


In [ ]:
px.box(_df, color="is_zz", y ="sugars_100g" ,x="pnns_groups_1")

In [ ]:
px.box(_df, x="nutrition_grade_fr", y ="fat_100g" )

In [ ]:
_df = df.loc[df.nutrition_grade_fr.notna()]
_df = _df.sort_values("nutrition_grade_fr")
px.box(_df, x="nutrition_grade_fr", y ="fat_100g" )

In [ ]:
df.product_name = df.product_name.fillna("unknown")

# df.product_name = df.product_name.apply(clean)
# df.product_name = df.product_name.apply(lambda i : i.lower())
df.product_name = df.product_name.str.lower()
df

In [ ]:
_df = df.copy()

_df = _df.loc[df.product_name.str.startswith("huile"), :]
_df = _df.loc[df.product_name.str.contains("olive"), :]

_df = _df.loc[_df.energy_100g.notna()]
_df

In [ ]:
_df = _df.loc[_df.energy_100g.notna()
              ]
_df

In [ ]:
px.box(_df.energy_100g)

In [ ]:
numeric_features = ["age", "fare"]
categorical_features = ["embarked", "sex", "pclass"]


# numeric_transformer = Pipeline(
#     steps=[("imputer", KNNImputer()), 
#            ("scaler", StandardScaler())]
# )
numeric_transformer = Pipeline(
    steps=[("passthrough", "passthrough"), 
        #    ("scaler", StandardScaler()), 
           ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        # ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor

In [ ]:
preprocessor

In [ ]:
pipe = Pipeline(
    [
        # ("sampler", RandomUnderSampler()),
        ("preprocessor", preprocessor)
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("estimator", LogisticRegression()),
    ]
)





y 
y_log

X_log  = mis au log certains features =)=> 
X_raw =  ans log feats


1 / modeles + simple > compiqué 

    * dummy 
    * regreslion lineaire que numérique pourie  (baseline )
    * gtravailler sur pipes diiférents / estifmateurs metatprametre y log sans log X raw X log 
    ==> 1 OU plusieurs modeles performats


2/ impact energy star score
    la version ou tu jette Energy star SCORERS
    La verison ou tu gardes enrgy star socre (==> KNNIMPUTER)

In [ ]:
1 / modeles + simple > compiqué 

    * dummy 
    * regreslion lineaire que numérique pourie  (baseline )
    * gtravailler sur pipes diiférents / estifmateurs metatprametre y log sans log X raw X log 
    ==> 1 OU plusieurs modeles performats


2/ impact energy star score
    la version ou tu jette Energy star SCORERS
    La verison ou tu gardes enrgy star socre (==> KNNIMPUTER)